In [1]:
## This script will be used to create response files for the care task
## I am not sure about if l is care or not though
## participants responded with either a left or right shift click... one of these is Yes for care and one is No for care

##  Acouple of notes I need to be aware of:
## 1. 628 is a nonsense code - I can remove these rows
## 2. I need to find a good way to protect against corrected answers
##   a. The stimuli are given a random interval any where between 1 and 1.5 seconds
##   in order to protect against this I need to find the windows between presentations
## 3. The goal is to determine how efficient these folks are at processing - so I will use the corrected answer
## it changes the answer 

In [2]:
#Stim Trigs (Care)
#3 (Crying)
#4 (Unhappy)
#5 (Neutral)
#6 (Happy)

#Response Trigs (Care)
#1 ("L Shift")
#2 ("R Shift")


In [3]:
import os
import pandas as pd
import numpy as np
from sklearn import metrics

In [5]:
## Run a test example here
df = pd.read_csv("/home/arosen/Documents/bbmcPersonal/eegBehavioralData/Care/117-32_Parent_Care-export.evt", sep='\t', header=0)

In [6]:
## First remove all 628 values from the df
df = df[-df["TriNo"].isin(["628"])]

In [8]:
## Now isolate the stimuli presentation from the answers
df_stim = df[df['TriNo'].isin(['3', '4', '5', '6'])]
df_stim.head(5)

,Tmu,Code,TriNo,Comnt
0,4179000,1,6,Trigger - DIN8
3,6044000,1,5,Trigger - DI64
5,7227000,1,4,Trigger - DI32
7,8725000,1,6,Trigger - DIN8
10,9825000,1,3,Trigger - DI16


In [9]:
%qtconsole

In [40]:
## Do the same for the reponses
df_resp = df[df['TriNo'].isin(['1', '2'])]
df_resp.head(5)

,Tmu,Code,TriNo,Comnt
2,5677000,1,2,Trigger - DI12
4,6893000,1,2,Trigger - DI12
6,8237000,1,1,Trigger - DI10
9,9325000,1,2,Trigger - DI12
11,10821000,1,1,Trigger - DI10


In [41]:
## Now I need to isolate the values that are inbetween intervals
## every stim should have 1 resp.. but there may be two for some
## This will be done in a for loop
## 118-314_OutResponse.csv 146

## and I will print the values that have multiple responses
## then I will have to take the second or first.... to be determined
row_vals = df_stim.shape[0]-1
## Now create an output data frame for everything to land into
d = pd.DataFrame([])
for x in range(row_vals):
    ## First identify the range
    lower_val = df_stim['Tmu         '].iloc[x]
    if x == df_stim.shape[0]:
        upper_val = df_stim['Tmu         '].iloc[x] + 1500000
    else:
        upper_val = df_stim['Tmu         '].iloc[x+1]
    ## Now check to see if the values are inbetween the interval in the resp
    if df_resp[df_resp['Tmu         '].between(lower_val, upper_val)].shape[0] > 1:
        print("Case at " ,x)
        ## Grab the last response
        row_index = df_resp[df_resp['Tmu         '].between(lower_val, upper_val)].shape[0] - 1
        #row_index = 0
        out_row = df_resp[df_resp['Tmu         '].between(lower_val, upper_val)].iloc[row_index]
    
    elif df_resp[df_resp['Tmu         '].between(lower_val, upper_val)].shape[0] == 0:
        print("Null response @ ", x)
        out_row = {'col1': [lower_val], 'col2': [np.nan], 'col3': [np.nan], 'col4': [np.nan]}
        out_row = pd.DataFrame(out_row)
        out_row.columns = df_stim.columns
    
    ## Now attach these all to the output
    else:
        out_row = df_resp[df_resp['Tmu         '].between(lower_val, upper_val)]
        
    input_stim = df_stim.iloc[x]
    d = d.append(input_stim)
    d = d.append(out_row)


Null response @  9
Null response @  13
Null response @  226
Null response @  314


In [42]:
## Now grab all of the even values - these are the stimuli
# We are going through this for two reasons:
# 1. AFGR is terrible at python programming
# 2. We want to get the differece in presentation and answer
#  to do 2 I am going to attach these togerther and calculate the differnece
df_stim = d[d['TriNo'].isin(['3', '4', '5', '6'])]
df_ans = d[-d['TriNo'].isin(['3', '4', '5', '6'])]
df_c = pd.concat([df_stim.reset_index(drop=True), df_ans.reset_index(drop=True)], axis=1)

In [43]:
## Now calculate the difference in time
df_c.columns = ['Index','CommentStim','TimeStim','TriNoStim','CodeAns','CommentAns', 'TimeAns', 'TriNoAns']

In [45]:
## Now create the difference between timeStim and TimeAns
df_c['timeToAnswer'] = df_c['TimeAns'] - df_c['TimeStim']

In [ ]:
## Now create a confusion matrix
#print(metrics.classification_report(df_c['TriNoStim'], df_c['TriNoAns']))

In [50]:
#!/usr/bin/env python
# coding: utf-8

# In[98]:


## This script will be used to create the confusion matrix
## as well as create some performance efficiency metrics

##  Acouple of notes I need to be aware of:
## 1. 628 is a nonsense code - I can remove these rows
## 2. I need to find a good way to protect against corrected answers
##   a. The stimuli are given a random interval any where between 1 and 1.5 seconds
##   in order to protect against this I need to find the windows between presentations
## 3. The goal is to determine how efficient these folks are at processing - so I will use the corrected answer
## it changes the answer 


# In[243]:


#Stim Trigs (ID)
#1 (Crying)
#6 (Unhappy)
#7 (Neutral)
#8 (Happy)

#Response Trigs (ID)
#2 ("A")
#3 ("S")
#4 ("K")
#5 ("L")

#Stim Trigs (Care)
#3 (Crying)
#4 (Unhappy)
#5 (Neutral)
#6 (Happy)

#Response Trigs (Care)
#1 ("L Shift")
#2 ("R Shift")


# In[244]:


import glob, os
import pandas as pd
import numpy as np
from sklearn import metrics


# In[245]:


## Run a test example here
os.chdir("/home/arosen/Documents/bbmcPersonal/eegBehavioralData/newData/eeg2/")
for df in glob.glob("*.evt"):
    print(df)
    ## Now create the output name while we have the string in memory
    out_name = "/home/arosen/Documents/bbmcPersonal/eegBehavioralData/CARE_mod/" + df.replace("#", "").partition("_")[0] + "_OutResponse.csv"
    df = pd.read_csv(df, sep='\t', header=0)

    
    
    ## First remove all 628 values from the df
    #df = df[-df["TriNo"].isin(["628"])]
    
    
    # In[247]:
    
    
    ## Now isolate the stimuli presentation from the answers
    df_stim = df[df['TriNo'].isin(['3', '4', '5', '6'])]
    
    
    # In[248]:
    
    
    ## Do the same for the reponses
    df_resp = df[df['TriNo'].isin(['1','2'])]
    
    
    # In[249]:
    
    
    ## Now I need to isolate the values that are inbetween intervals
    ## every stim should have 1 resp.. but there may be two for some
    ## This will be done in a for loop
    ## and I will print the values that have multiple responses
    ## then I will have to take the second or first.... to be determined
    row_vals = df_stim.shape[0] - 1
    ## Now create an output data frame for everything to land into
    d = pd.DataFrame([])
    for x in range(row_vals):
        ## First identify the range
        lower_val = df_stim['Tmu         '].iloc[x]
        if x == df_stim.shape[0]-1:
            upper_val = df_stim['Tmu         '].iloc[x] + 1500000
            print("Final Row")
        else:
            upper_val = df_stim['Tmu         '].iloc[x+1]
        ## Now check to see if the values are inbetween the interval in the resp
        if df_resp[df_resp['Tmu         '].between(lower_val, upper_val)].shape[0] > 1:
            #print("Case at " ,x)
            ## Grab the last response
            row_index = df_resp[df_resp['Tmu         '].between(lower_val, upper_val)].shape[0] - 1
            #row_index = 0
            out_row = df_resp[df_resp['Tmu         '].between(lower_val, upper_val)].iloc[row_index]
        
        elif df_resp[df_resp['Tmu         '].between(lower_val, upper_val)].shape[0] == 0:
            #print("Null response @ ", x)
            out_row = {'col1': [lower_val], 'col2': [np.nan], 'col3': [np.nan], 'col4': [np.nan]}
            out_row = pd.DataFrame(out_row)
            out_row.columns = df_stim.columns
        
        ## Now attach these all to the output
        else:
            out_row = df_resp[df_resp['Tmu         '].between(lower_val, upper_val)]
            
        input_stim = df_stim.iloc[x]
        input_stim['numOfResponses'] = df_resp[df_resp['Tmu         '].between(lower_val, upper_val)].shape[0]
        ## Now add a column with the number of responses
        d = d.append(input_stim)
        d = d.append(out_row)


    ## Now grab all of the even values - these are the stimuli
    # We are going through this for two reasons:
    # 1. AFGR is terrible at python programming
    # 2. We want to get the differece in presentation and answer
    #  to do 2 I am going to attach these togerther and calculate the differnece
    df_stim = d[d['TriNo'].isin(['3', '4', '5', '6'])]
    df_ans = d[-d['TriNo'].isin(['3', '4', '5', '6'])]
    df_c = pd.concat([df_stim.reset_index(drop=True), df_ans.reset_index(drop=True)], axis=1)


    ## Now calculate the difference in time
    df_c.columns = ['Index','CommentStim','TimeStim','TriNoStim','noOfResposesStim','CodeAns','CommentAns', 'TimeAns', 'TriNoAns', 'numOfResponsesAns']



    ## Now create the difference between timeStim and TimeAns
    df_c['timeToAnswer'] = df_c['TimeAns'] - df_c['TimeStim']

    ## Now write the output
    df_c.to_csv(out_name)








250-10#_Parent_Care.evt


/home/arosen/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:123: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


117-115_Parent_Care-export.evt
248-23#_Parent_Care.evt
118-286_Parent_Care-export.evt
222-34_Parent_Care-export.evt
255-23#_Parent_Care.evt
117-59_Parent_Care-export.evt
118-103_Parent_Care-export.evt
119-102_Parent_Care-export.evt
222-36_Parent_Care-export.evt
119-112_Parent_Care-export.evt
117-50#_Parent_Care-export.evt
118-172_Parent_Care-export.evt
118-292_Parent_Care-export.evt
118-300_Parent_Care-export.evt
118-232_Parent_Care.evt
117-71_Parent_Care-export.evt
255-22#_Parent_Care.evt
250-23#_Parent_Care.evt
117-34_Parent_Care-export.evt
118-191_Parent_Care-export.evt
118-132_Parent_Care-export.evt
255-44#_Parent_Care.evt
248-10#_Parent_Care.evt
117-46#_Parent_Care-export.evt
250-26#_Parent_Care.evt
117-77#_Parent_Care-export.evt
118-128_Parent_Care-export.evt
118-143_Parent_Care-export.evt
250-36#_Parent_Care.evt
118-198_Parent_Care-export.evt
118-107_Parent_Care-export.evt
222-13#_Parent_Care.evt
255-33#_Parent_Care.evt
255-64#_Parent_Care.evt
121-4_Parent_Care-export.evt
118-31